In [3]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS 
import numpy as np
from matplotlib import pyplot as plt

In [4]:
import numpy as np
df = pd.read_csv(
    "C:\\Users\\Kathe\\CitibikeNYC\\3.3_Include_Trips.csv",
    index_col=0,
    low_memory=False
)

In [5]:
df.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'member_casual', 'date', 'avgTemp',
       'trips_per_day'],
      dtype='object')

In [13]:
import pandas as pd

# Assuming 'df' is your existing DataFrame with the mentioned columns

# Create a long-form DataFrame for Kepler.gl
df_start = df[['start_lat', 'start_lng', 'started_at', 'start_station_name', 'member_casual']].copy()
df_start.rename(columns={
    'start_lat': 'latitude',
    'start_lng': 'longitude',
    'started_at': 'timestamp',
    'start_station_name': 'station_name'
}, inplace=True)
df_start['trip_point'] = 'start'

df_end = df[['end_lat', 'end_lng', 'ended_at', 'end_station_name', 'member_casual']].copy()
df_end.rename(columns={
    'end_lat': 'latitude',
    'end_lng': 'longitude',
    'ended_at': 'timestamp',
    'end_station_name': 'station_name'
}, inplace=True)
df_end['trip_point'] = 'end'

# Add unique trip ID to link start and end points
df_start['trip_id'] = df.index
df_end['trip_id'] = df.index

# Concatenate both start and end dataframes to form a long format
df_long = pd.concat([df_start, df_end], ignore_index=True)

# Ensure timestamps are formatted correctly for Kepler.gl
df_long['timestamp'] = pd.to_datetime(df_long['timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')

# Save to CSV for Kepler.gl upload
df_long.to_csv('citibike_kepler_ready.csv', index=False)

print("Data successfully transformed and saved to citibike_kepler_ready.csv")

Data successfully transformed and saved to citibike_kepler_ready.csv


In [7]:
# Create a value and group by start and end station
df['value'] = 1
df_group = df.groupby(['started_at', 'ended_at'])['value'].count().reset_index()

In [8]:
print(df_group['value'].sum())
print(df.shape)

298379
(298379, 16)


In [11]:
df_group['started_at'] = df_group['started_at'].astype(str)
df_group['ended_at'] = df_group['ended_at'].astype(str)

In [13]:
df_group.dtypes

started_at    object
ended_at      object
value          int64
dtype: object

In [15]:
df_group['started_at'] = df_group['started_at'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df_group['ended_at'] = df_group['ended_at'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [17]:
print(df_group.applymap(type).nunique())

started_at    1
ended_at      1
value         1
dtype: int64


In [19]:
df_group['started_at'] = df_group['started_at'].str.strip()
df_group['ended_at'] = df_group['ended_at'].str.strip()

In [21]:
df_group.dropna(subset=['started_at', 'ended_at'], inplace=True)